# Transform

Testing ground for service call data transformation

In [1]:
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

import os
from dotenv import load_dotenv
from datetime import datetime

import folium
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql import Window
from pyspark.sql import types

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 10:10:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pq_path = Path("../tests/resources/SR2020.parquet")
df = spark.read.parquet(str(pq_path))
df.printSchema()

root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)



In [ ]:
# stop current SparkContext before creating a new one
spark.stop()

## Cloud Setup

Read from cloud storage using `gcs-connector-hadoop3` jar

Before starting a new context, I had to restart the kernel for spark to recognize the gcs-connector jar

In [31]:
# !mkdir ../data/lib
# !gsutil cp gs://hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar  \
#     ../data/lib/gcs-connector-hadoop3-latest.jar
# !curl -O https://github.com/GoogleCloudDataproc/hadoop-connectors/releases/download/v2.2.11/gcs-connector-hadoop3-2.2.11-shaded.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [2]:
conf = (
    SparkConf()
    .setMaster("local[*]")
    .setAppName("test_cloud")
    .set("spark.jars", "../data/lib/gcs-connector-hadoop3-latest.jar")
)
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set(
    "fAbstractFileSystem.gimpl", "com.google.cloud.hadoop.fgcGoogleHadoopFS"
)
hadoop_conf.set("fgimpl", "com.google.cloud.hadoop.fgcGoogleHadoopFileSystem")
spark = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()
load_dotenv()
DATA_LAKE = os.getenv("DATA_LAKE")

23/03/25 13:47:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
gcs_path = f"gs://{DATA_LAKE}/raw/pq/sr2023.parquet"
print(f"gcs_path: {gcs_path}")
df_gcs = spark.read.parquet(gcs_path)
df_gcs.printSchema()

gcs_path: gs://service-calls-data-lake/raw/pq/sr2023.parquet


root
 |-- Status: string (nullable = true)
 |-- First 3 Chars of Postal Code: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Service Request Type: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- Section: string (nullable = true)
 |-- ward_name: string (nullable = true)
 |-- ward_id: byte (nullable = true)
 |-- creation_datetime: timestamp (nullable = true)



## Transformations

How should we model our service call data?

Most important fields:

- Service Request Type
- First 3 Chars of Postal Code
- ward_name
- datetime

Ideas:

- most common types per season?
    - top 5?
- most common types by ward/FSA?
    - least common may not be as interesting, due to the high granularity in how types are categorized
- highest frequency ward/FSA by service types, i.e. which ward has the most/least amount of noise complaints?
    - for each service type (count > 100), top 3 most common ward?
    - % of all requests for each ward?
- combine with population per FSA and find requests per capita?
- distributions of the all/top 80% of request types
    - filter by ward/FSA/season?

Division and section could be queried against to show which municipal entity receive the most requests, although that would highly correlate with the type of requests

In [3]:
# for testing
load_dotenv()
DATA_LAKE = os.getenv("DATA_LAKE")
gcs_path = f"gs://{DATA_LAKE}/raw/pq/SR2021.parquet"
df = spark.read.parquet(gcs_path)
print(f"row count: {df.count()}")

row count: 323880


In [16]:
try:
    df.corr("Service Request Type", "Division")
except Exception as e:
    print(e)

requirement failed: Currently correlation calculation for columns with dataType string not supported.


### Most common request type by ward

In [5]:
# request type count by ward
df_ward_type = (
    df.filter(df.ward_name.isNotNull())
    .select("creation_datetime", "ward_name", "Service Request Type")
    .withColumnRenamed("Service Request Type", "service_request_type")
    .groupBy(["ward_name", "service_request_type"])
    .count()
)
# most common request type's count by ward
df_ward_max = (
    df_ward_type.groupBy("ward_name")
    .max("count")
    .withColumnRenamed("max(count)", "max_count")
)

# trying to join them get get most common type by ward
df_ward = (
    df_ward_max.join(df_ward_type, col("max_count") == col("count"), "inner")
    .select("df_ward_max.ward_name", "Service Request Type", "count")
    .orderBy("ward_name")
)
df_ward.show(n=30, truncate=40)

+------------------------+---------------------------------------+-----+
|               ward_name|                   Service Request Type|count|
+------------------------+---------------------------------------+-----+
|       Beaches-East York|                       CADAVER WILDLIFE|  788|
|               Davenport|Residential: Bin: Repair or Replace Lid| 1141|
|         Don Valley East|                     Property Standards|  349|
|        Don Valley North|                                 Zoning|  413|
|        Don Valley North|Residential: Bin: Repair or Replace Lid|  413|
|         Don Valley West|                        General Pruning|  575|
|       Eglinton-Lawrence|Residential: Bin: Repair or Replace Lid|  984|
|        Etobicoke Centre|                        General Pruning|  845|
|         Etobicoke North|                     Property Standards|  517|
|     Etobicoke-Lakeshore|                        General Pruning|  913|
|Humber River-Black Creek|                     Prop

Seeing some repeats of ward name due to counts of different request types being equal; is that a coincidence or is that really true?

In [8]:
df_ward_max.orderBy("ward_name").count()

25

In [9]:
df_ward.count()

29

Using a `Window` instead, as explained in [this SOF post](https://stackoverflow.com/questions/48829993/groupby-column-and-filter-rows-with-maximum-value-in-pyspark)

- find max service type by ward
- assign that value as a new column, according to ward
- filter for rows where count == max_count
- that row is the max_count service type

In [15]:
w = Window.partitionBy("ward_name")
df_ward_type.withColumn("max_count", F.max("count").over(w)).where(
    F.col("count") == F.col("max_count")
).drop("max_count").show(n=25, truncate=40)

+------------------------+---------------------------------------+-----+
|               ward_name|                   Service Request Type|count|
+------------------------+---------------------------------------+-----+
|       Beaches-East York|                       CADAVER WILDLIFE|  788|
|               Davenport|Residential: Bin: Repair or Replace Lid| 1141|
|         Don Valley East|                     Property Standards|  349|
|        Don Valley North|Residential: Bin: Repair or Replace Lid|  413|
|         Don Valley West|                        General Pruning|  575|
|       Eglinton-Lawrence|Residential: Bin: Repair or Replace Lid|  984|
|        Etobicoke Centre|                        General Pruning|  845|
|         Etobicoke North|                     Property Standards|  517|
|     Etobicoke-Lakeshore|                        General Pruning|  913|
|Humber River-Black Creek|                     Property Standards|  499|
|      Parkdale-High Park|Residential: Bin: Repair 

In [25]:
df_ward_type.filter(df_ward_type.ward_name == "Willowdale").orderBy(
    "count", ascending=False
).show(n=5)

+----------+--------------------+-----+
| ward_name|Service Request Type|count|
+----------+--------------------+-----+
|Willowdale|Long Grass and Weeds|  410|
|Willowdale|Residential: Bin:...|  386|
|Willowdale|     General Pruning|  375|
|Willowdale|    CADAVER WILDLIFE|  313|
|Willowdale| INJUR/DIST WILDLIFE|  285|
+----------+--------------------+-----+
only showing top 5 rows



Seems my first approach picked up some erroneous row Simply matching the max_count and hoping they were unique was perhaps not the best way to go about it.

## Window function

Characterized by `OVER (PARTITION BY ...)`, there are 3 types:

1. Aggregate
1. Ranking
1. Value

### Aggregate

Aggregates refer to:

- AVG
- MAX
- MIN
- SUM
- COUNT

This can be a replacement for `GROUP BY`, if we specify that group in the `OVER` clause. But unlike `GROUPBY.COUNT` which collapses the table into those groups, window assigns that value to each row of that dataset.

In SQL for above query:

In [4]:
df.createOrReplaceTempView("df_sql_view")

Part one - creating table with count of each type for each ward

In [61]:
query = """
    SELECT
        ward_name ward,
        `Service Request Type` type,
        COUNT(1) count
    FROM
        df_sql_view
    GROUP BY ward_name, `Service Request Type`
    HAVING ward_name IS NOT NULL
    ORDER BY count DESC
"""
df_type_count = spark.sql(query)
df_type_count.createOrReplaceTempView("df_type_count")
df_type_count.show()

+--------------------+--------------------+-----+
|                ward|                type|count|
+--------------------+--------------------+-----+
|           Davenport|Residential: Bin:...| 1141|
|   Eglinton-Lawrence|Residential: Bin:...|  984|
|  Parkdale-High Park|Residential: Bin:...|  971|
| Etobicoke-Lakeshore|     General Pruning|  913|
|  Toronto-St. Paul's|Residential: Bin:...|  895|
|    Etobicoke Centre|     General Pruning|  845|
|    Toronto-Danforth|     General Pruning|  818|
| University-Rosedale|Residential: Bin:...|  810|
| Etobicoke-Lakeshore| INJUR/DIST WILDLIFE|  799|
|   Beaches-East York|    CADAVER WILDLIFE|  788|
|   Beaches-East York| INJUR/DIST WILDLIFE|  784|
|   Beaches-East York|Residential: Bin:...|  774|
|   Eglinton-Lawrence|     General Pruning|  751|
|   Beaches-East York|     General Pruning|  737|
|    Toronto-Danforth|    CADAVER WILDLIFE|  728|
|   York South-Weston|  Property Standards|  716|
|Scarborough South...|  Property Standards|  701|


Part two - using common table expression (CTE) to add a column with max(count) of each ward, and query against it to find the max count service type

In [67]:
query = """
WITH type_max AS (
    SELECT 
        ward,
        type,
        count,
        MAX(count) OVER (PARTITION BY ward) maxcount
    FROM df_type_count
)
SELECT
    ward, type, count
FROM
    type_max
WHERE count == maxcount
"""
df_type_max = spark.sql(query)
df_type_max.show()

+--------------------+--------------------+-----+
|                ward|                type|count|
+--------------------+--------------------+-----+
|   Beaches-East York|    CADAVER WILDLIFE|  788|
|           Davenport|Residential: Bin:...| 1141|
|     Don Valley East|  Property Standards|  349|
|    Don Valley North|Residential: Bin:...|  413|
|     Don Valley West|     General Pruning|  575|
|   Eglinton-Lawrence|Residential: Bin:...|  984|
|    Etobicoke Centre|     General Pruning|  845|
|     Etobicoke North|  Property Standards|  517|
| Etobicoke-Lakeshore|     General Pruning|  913|
|Humber River-Blac...|  Property Standards|  499|
|  Parkdale-High Park|Residential: Bin:...|  971|
|  Scarborough Centre|Residential: Bin:...|  668|
|   Scarborough North|Residential: Bin:...|  531|
|Scarborough South...|  Property Standards|  701|
|Scarborough-Aginc...|     General Pruning|  428|
|Scarborough-Guild...| INJUR/DIST WILDLIFE|  484|
|Scarborough-Rouge...|     General Pruning|  687|


Putting it all together. CTE used to construct a table with `maxcount` column to query against so that we can use it in the `WHERE` statement; `WHERE` cannot accept windows

In [8]:
window_query = """
with type_count as (
    SELECT
        ward_name ward,
        `Service Request Type` type,
        COUNT(1) count
    FROM
        df_sql_view
    GROUP BY ward_name, `Service Request Type`
    HAVING ward_name IS NOT NULL
),
type_max as (
    SELECT 
        ward,
        type,
        count,
        MAX(count) OVER (PARTITION BY ward) maxcount
    FROM type_count
)
SELECT
    ward, type, count, row_number() over(ORDER BY type) row
FROM
    type_max
WHERE count == maxcount
ORDER BY type
"""
df_sql = spark.sql(window_query).show(n=25, truncate=40)

23/03/22 14:47:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/22 14:47:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/22 14:47:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---------

### Most common ward by request type

Inverse of the previous question - which neighborhood makes the most calls for each request type?

In [12]:
ward_query = """
with type_count as (
    SELECT
        ward_name ward,
        `Service Request Type` type,
        COUNT(1) count
    FROM
        df_sql_view
    GROUP BY ward_name, `Service Request Type`
    HAVING ward_name IS NOT NULL
),
ward_max as (
    SELECT 
        ward,
        type,
        count,
        MAX(count) OVER (PARTITION BY type) maxcount
    FROM type_count
)
SELECT
    type, ward, count
FROM ward_max
WHERE maxcount = count and count > 100
"""
df_ward_by_type = spark.sql(ward_query).show()
# df_ward_by_type

+--------------------+--------------------+-----+
|                type|                ward|count|
+--------------------+--------------------+-----+
|All / Hazardous W...|  Parkdale-High Park|  656|
|Bin Investigation...|           Davenport|  511|
|Boulevard - Ploug...|Scarborough-Guild...|  102|
|Boulevards - Dama...|    Etobicoke Centre|  194|
|By-Law Contravent...| Etobicoke-Lakeshore|  126|
|    CADAVER WILDLIFE|   Beaches-East York|  788|
|Catch Basin - Blo...|   Eglinton-Lawrence|  134|
|Comment / Suggestion|   Spadina-Fort York|  118|
|Complaint/Investi...|           Davenport|  179|
|Dispute SR Status...|   Eglinton-Lawrence|  211|
|      Dogs off Leash|   Spadina-Fort York|  131|
|Expressway requir...|   Spadina-Fort York|  122|
|     General Pruning| Etobicoke-Lakeshore|  913|
|General Tree Main...| Etobicoke-Lakeshore|  115|
|            Graffiti| University-Rosedale|  290|
|Gypsy Moth Contro...|    Don Valley North|  168|
| INJUR/DIST WILDLIFE| Etobicoke-Lakeshore|  799|


### Top 5 requests by season

Let's define seasons by datetime:

- spring: 3/21-6/21
- summer: 6/21-9/21
- fall: 9/21 - 11/30 (winter is long, okay)
- winter: 12/1 - 3/21

Execution:

- add season column via UDF
- for each season, count rows by service type
- since we want the top $n$ service type by season, we can't just use `max`
    - for each season, sort the count
    - get row numbers 1-5 for each season - use window
    
Alternative - SQL execution?

- CASE/WHEN/THEN to extract season
- `row_number() OVER(PARTITION BY season)`

#### Pyspark

In [6]:
# dt_samp, _ = df.randomSplit([0.1,0.9])
dt_samp = df.sample(0.005).select("creation_datetime")
dt_samp.createOrReplaceTempView("dt_samp")
dt_samp.show()

+-------------------+
|  creation_datetime|
+-------------------+
|2021-01-01 09:45:45|
|2021-01-02 09:34:10|
|2021-01-02 11:07:43|
|2021-01-02 12:47:47|
|2021-01-02 13:53:55|
|2021-01-02 16:20:38|
|2021-01-03 09:12:28|
|2021-01-03 12:49:48|
|2021-01-03 18:55:33|
|2021-01-04 11:36:54|
|2021-01-04 12:01:47|
|2021-01-04 17:18:47|
|2021-01-04 20:58:35|
|2021-01-05 08:13:23|
|2021-01-05 09:29:38|
|2021-01-05 10:21:36|
|2021-01-05 11:40:57|
|2021-01-05 12:22:56|
|2021-01-05 13:30:11|
|2021-01-06 12:25:47|
+-------------------+
only showing top 20 rows



In [7]:
dt_samp.count()

1649

In [8]:
dt1 = dt_samp.head(1)
type(dt1[0].creation_datetime)
dtval = dt1[0].creation_datetime
dtval.year
spring = datetime(dtval.year, month=3, day=21)
dtval < spring

True

In [9]:
def extract_season(creation_time: datetime) -> str:
    """Extracts season from the datetime field

    Use as spark UDF
    """
    m = creation_time.month
    d = creation_time.day
    if m < 3 or (m == 3 and d < 21) or m > 11:
        return "winter"
    elif m < 6 or (m == 6 and d < 21):
        return "spring"
    elif m < 9 or (m == 9 and d < 21):
        return "summer"
    else:
        return "fall"

In [10]:
# instantiate spark UDF
extract_season_udf = F.udf(extract_season, returnType=types.StringType())

In [11]:
# call UDF on the creation_datetime column
dt_samp.withColumn("season", extract_season_udf(dt_samp.creation_datetime)).sample(
    0.02
).show(n=40)

+-------------------+------+
|  creation_datetime|season|
+-------------------+------+
|2021-01-16 13:04:01|winter|
|2021-01-19 18:30:09|winter|
|2021-02-23 13:31:35|winter|
|2021-03-25 11:49:37|spring|
|2021-04-04 10:39:06|spring|
|2021-04-21 12:51:34|spring|
|2021-04-23 15:40:24|spring|
|2021-05-11 11:46:21|spring|
|2021-06-03 14:56:26|spring|
|2021-06-04 17:00:54|spring|
|2021-06-10 09:11:18|spring|
|2021-06-11 15:36:45|spring|
|2021-06-14 22:29:32|spring|
|2021-06-18 18:13:40|spring|
|2021-06-23 08:10:08|summer|
|2021-06-23 11:03:17|summer|
|2021-06-30 14:10:12|summer|
|2021-07-01 09:05:45|summer|
|2021-07-16 16:11:03|summer|
|2021-07-17 19:29:50|summer|
|2021-07-21 13:59:00|summer|
|2021-08-05 15:40:58|summer|
|2021-08-30 09:03:25|summer|
|2021-09-02 09:23:01|summer|
|2021-10-20 19:22:25|  fall|
|2021-10-28 13:21:34|  fall|
+-------------------+------+



In [37]:
# Specify in window that `count` must be ordered as descending
# by referencing the COLUMN and using the `.desc()` method
w = Window.partitionBy("season").orderBy(F.col("count").desc())
df_season_type = (
    df.sample(0.1)
    .withColumnRenamed("Service Request Type", "service_request_type")
    .withColumn("season", extract_season_udf(df.creation_datetime))
    .groupby("season", "service_request_type")
    .count()
    .withColumn("seasonal_rank", F.row_number().over(w))
    .filter(F.col("seasonal_rank") < 4)
    .show(truncate=40)
)

+------+---------------------------------------+-----+-------------+
|season|                   service_request_type|count|seasonal_rank|
+------+---------------------------------------+-----+-------------+
|  fall|Residential: Bin: Repair or Replace Lid|  186|            1|
|  fall|                     Property Standards|  179|            2|
|  fall|                       CADAVER WILDLIFE|  167|            3|
|spring|                        General Pruning|  448|            1|
|spring|Residential: Bin: Repair or Replace Lid|  446|            2|
|spring|                    INJUR/DIST WILDLIFE|  419|            3|
|summer|                        General Pruning|  614|            1|
|summer|Residential: Bin: Repair or Replace Lid|  481|            2|
|summer|                       CADAVER WILDLIFE|  451|            3|
|winter|                    INJUR/DIST WILDLIFE|  319|            1|
|winter|                                 Zoning|  309|            2|
|winter|Residential: Bin: Repair o

#### Rank vs Dense Rank vs Row

Differs in how they handle ties:

|col1|col2|rank|dense_rank|row_number|
|----|----|----|----------|----------|
|   a|  10|   1|         1|         1|
|   a|  10|   1|         1|         2|
|   a|  20|   3|         2|         3|

`row_number` will always give unique IDs

#### SQL CASE/WHEN

Extracting season with `CASE/WHEN/THEN/ELSE` in SQL

In [ ]:
season_query = """
    SELECT
        creation_datetime,
        CASE
            WHEN MONTH(creation_datetime) < 3 OR MONTH(creation_datetime) > 11 OR
            MONTH(creation_datetime) = 3 AND DAY(creation_datetime) < 21 THEN 'winter'
            WHEN MONTH(creation_datetime) < 6 OR MONTH(creation_datetime) = 6 AND DAY(creation_datetime) < 21 THEN 'spring'
            WHEN MONTH(creation_datetime) < 9 OR MONTH(creation_datetime) = 9 AND DAY(creation_datetime) < 21 THEN 'summer'
            ELSE 'fall'
        END as season,
        count(1) count
    FROM
        dt_samp
    GROUP BY season
    TABLESAMPLE(0.1 PERCENT)
"""
sql_season = spark.sql(season_query)
sql_season.sample(0.02).show(n=40)

1. First CTE `season_type_count` creates the count column by aggregating season and types
1. Second CTE `season_rank_table` creates the `seasonal_rank` column using window function `row_number()`
1. Final query filters the `seasonal_rank` column. Window functions can only be used in `SELECT` clauses, and so if we want to filter the window function return values, we create the CTE to query against.

In [39]:
season_sort = """
    WITH season_type_count AS (
        SELECT
            CASE
                WHEN MONTH(creation_datetime) < 3 OR MONTH(creation_datetime) > 11 THEN 'winter'
                WHEN MONTH(creation_datetime) < 6 THEN 'spring'
                WHEN MONTH(creation_datetime) < 9 THEN 'summer'
                ELSE 'fall'
            END season,
            `Service Request Type` service_request_type,
            COUNT(1) count
        FROM
            df_sql_view
        TABLESAMPLE (10 PERCENT)
        GROUP BY season, `Service Request Type`
    ),
    season_rank_table AS(
        SELECT
            season, 
            service_request_type,
            count,
            ROW_NUMBER() OVER(PARTITION BY season ORDER BY count DESC) seasonal_rank
        FROM season_type_count
    )
    SELECT *
    FROM season_rank_table
    WHERE seasonal_rank < 4
"""
spark.sql(season_sort).show()

+------+--------------------+-----+-------------+
|season|service_request_type|count|seasonal_rank|
+------+--------------------+-----+-------------+
|  fall|  Property Standards|  313|            1|
|  fall|Residential: Bin:...|  302|            2|
|  fall|    CADAVER WILDLIFE|  282|            3|
|spring|Residential: Bin:...|  434|            1|
|spring| INJUR/DIST WILDLIFE|  383|            2|
|spring|              Zoning|  347|            3|
|summer|     General Pruning|  665|            1|
|summer|Residential: Bin:...|  518|            2|
|summer|  Property Standards|  453|            3|
|winter|Residential: Bin:...|  213|            1|
|winter|              Zoning|  210|            2|
|winter| INJUR/DIST WILDLIFE|  204|            3|
+------+--------------------+-----+-------------+



### Add proportion when displaying the rank

- Calculate new column `total` using the window function over `count`
- Another column `percentage` from `count / total`
- carry `percentage` to the final table
- to format `percentage`, need to query it from another table; can't do `CAST(count / subtotal ...)`

In [43]:
season_sort = """
    WITH season_type_count AS (
        SELECT
            CASE
                WHEN MONTH(creation_datetime) < 3 OR MONTH(creation_datetime) > 11 THEN 'winter'
                WHEN MONTH(creation_datetime) < 6 THEN 'spring'
                WHEN MONTH(creation_datetime) < 9 THEN 'summer'
                ELSE 'fall'
            END AS season,
            `Service Request Type` AS service_request_type,
            COUNT(1) AS count
        FROM
            df_sql_view
        TABLESAMPLE (5 PERCENT)
        GROUP BY season, `Service Request Type`
    ),
    season_rank_table AS(
        SELECT
            season, 
            service_request_type,
            count,
            SUM(count) OVER(PARTITION BY season) AS subtotal,
            ROW_NUMBER() OVER(PARTITION BY season ORDER BY count DESC) AS seasonal_rank,
            count / SUM(count) OVER(PARTITION BY season) AS percentage
        FROM season_type_count
    )
    SELECT 
        season, 
        service_request_type,
        count,
        seasonal_rank,
        CAST(percentage AS DECIMAL(4, 3))
    FROM season_rank_table
    WHERE seasonal_rank < 4
    ORDER BY season, seasonal_rank ASC;
"""
spark.sql(season_sort).show()

+------+--------------------+-----+--------+-------------+----------+
|season|service_request_type|count|subtotal|seasonal_rank|percentage|
+------+--------------------+-----+--------+-------------+----------+
|  fall|  Property Standards|  151|    3413|            1|     0.044|
|  fall|     General Pruning|  133|    3413|            2|     0.039|
|  fall|Residential: Bin:...|  127|    3413|            3|     0.037|
|spring| INJUR/DIST WILDLIFE|  226|    4864|            1|     0.046|
|spring|Residential: Bin:...|  220|    4864|            2|     0.045|
|spring|              Zoning|  184|    4864|            3|     0.038|
|summer|     General Pruning|  340|    5711|            1|     0.060|
|summer|Residential: Bin:...|  251|    5711|            2|     0.044|
|summer|    CADAVER WILDLIFE|  219|    5711|            3|     0.038|
|winter|Residential: Bin:...|  105|    2417|            1|     0.043|
|winter|              Zoning|  103|    2417|            2|     0.043|
|winter| INJUR/DIST 

### Most common types by ward - top 5

Instead of only returning the most common (max), return the top 5 most common types per ward, using the window function, and include the proportions

- Filter by substring - use `WHERE name LIKE 'str_pattern'`
    - `%` zero or more
    - `_` one
    - `[abc]` any of a, b, or c
    - `[^abc]` matches all char that are *not* a, b, or c
    - `[a-z]` matches all single char in that range, i.e. all lower case letters

In [50]:
ward_top_query = """
with ward_count as (
    SELECT
        ward_name ward,
        `Service Request Type` type,
        COUNT(1) count
    FROM
        df_sql_view
    GROUP BY ward_name, `Service Request Type`
    HAVING ward_name IS NOT NULL
),
ward_rank_table as (
    SELECT
        ward,
        type,
        count,
        ROW_NUMBER() OVER(PARTITION BY ward ORDER BY count DESC) ward_rank,
        count / SUM(count) OVER(PARTITION BY ward) percentage
    FROM
        ward_count
)
SELECT
    ward,
    type,
    count,
    ward_rank,
    CAST(percentage AS DECIMAL(4,3))
FROM ward_rank_table
WHERE ward_rank < 4 AND ward LIKE 'Scar%'
ORDER BY ward, ward_rank
"""
spark.sql(ward_top_query).show(truncate=40)

+----------------------+---------------------------------------+-----+---------+----------+
|                  ward|                                   type|count|ward_rank|percentage|
+----------------------+---------------------------------------+-----+---------+----------+
|    Scarborough Centre|Residential: Bin: Repair or Replace Lid|  668|        1|     0.055|
|    Scarborough Centre|                     Property Standards|  484|        2|     0.040|
|    Scarborough Centre|                       CADAVER WILDLIFE|  443|        3|     0.037|
|     Scarborough North|Residential: Bin: Repair or Replace Lid|  531|        1|     0.067|
|     Scarborough North|                        General Pruning|  377|        2|     0.048|
|     Scarborough North|Res / Organic Green Bin / Not Picked Up|  300|        3|     0.038|
| Scarborough Southwest|                     Property Standards|  701|        1|     0.047|
| Scarborough Southwest|                       CADAVER WILDLIFE|  588|        2|

### Top wards by request types

Same as above, but group by types instead of wards

In [56]:
types_top_query = """
with types_count as (
    SELECT
        ward_name ward,
        `Service Request Type` type,
        COUNT(1) count
    FROM
        df_sql_view
    GROUP BY ward_name, `Service Request Type`
    HAVING ward_name IS NOT NULL
),
types_rank_table as (
    SELECT
        ward,
        type,
        count,
        ROW_NUMBER() OVER(PARTITION BY type ORDER BY count DESC) type_rank,
        count / SUM(count) OVER(PARTITION BY type) percentage
    FROM
        types_count
    WHERE
        count > 100
)
SELECT
    type,
    ward,
    count,
    type_rank,
    CAST(percentage AS DECIMAL(4,3))
FROM types_rank_table
WHERE type_rank < 4 AND percentage > 0.1
ORDER BY type, type_rank
"""
spark.sql(types_top_query).show(truncate=40)

+---------------------+--------------------------------------+-----+---------+----------+
|                 ward|                                  type|count|type_rank|percentage|
+---------------------+--------------------------------------+-----+---------+----------+
|            Davenport|             Bin Investigation Request|  511|        1|     0.130|
|  University-Rosedale|             Bin Investigation Request|  411|        2|     0.105|
|   Parkdale-High Park|             Bin Investigation Request|  410|        3|     0.104|
|Scarborough-Guildwood|             Boulevard - Plough Damage|  102|        1|     1.000|
|     Etobicoke Centre|          Boulevards - Damaged Asphalt|  194|        1|     0.240|
|  Etobicoke-Lakeshore|          Boulevards - Damaged Asphalt|  148|        2|     0.183|
|    Eglinton-Lawrence|          Boulevards - Damaged Asphalt|  147|        3|     0.182|
|  Etobicoke-Lakeshore|           By-Law Contravention Invest|  126|        1|     0.275|
|    Beach